In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import glob
from frosting_scene.gs_model import GaussianSplattingWrapper
from frosting_scene.frosting_model import load_frosting_model
from PIL import Image

torch.cuda.set_device('cuda:0')
device = torch.device(torch.cuda.current_device())

Using original 3DGS rasterizer from Inria.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/mnt/homeGPU/aruiz/frosting/lib/python3.9/site-packages/torch/utils/cpp_extension.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]


Using original 3DGS rasterizer from Inria.


# Source path

In [ ]:
# Path to your COLMAP dataset. You can change the path to the dataset you want to use.
source_path= './own_data/stick/'
if source_path[-1] != '/':
    source_path += '/'
your_scene_name = source_path.split('/')[-2]

# Path to the vanilla 3DGS output directory. You can change the path to the directory you want to use.
gs_output_dir = f'./output/vanilla_gs/{your_scene_name}/'

# Path to the Frosting model. You can change the path to the model you want to use.
frosting_dir = f'./output/refined_frosting/{your_scene_name}'
frosting_path = os.path.join(frosting_dir, os.listdir(frosting_dir)[0], '15000.pt')

print("Path to the COLMAP dataset: ", source_path, sep='\n')
print("Path to the vanilla 3DGS output directory: ", gs_output_dir, sep='\n')
print("Path to the Frosting model: ", frosting_path, sep='\n')

# Vanilla GS Model

In [ ]:
eval_split = True
load_gt_images = True
white_background = False
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]

gs_model = GaussianSplattingWrapper(
    source_path=source_path,
    output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
    iteration_to_load=7000,
    load_gt_images=load_gt_images,
    eval_split=eval_split,
    eval_split_interval=8,
    background=bg_color,
    white_background=white_background,
    remove_camera_indices=[],
)

# Load Frosting

In [ ]:
frosting = load_frosting_model(frosting_path, nerfmodel=gs_model)

print(f"Number of vertices in base mesh: {len(frosting._shell_base_verts)}")
print(f"Number of faces in base mesh: {len(frosting._shell_base_faces)}")
print(f"Number of Gaussians in the frosting layer: {len(frosting._bary_coords)}")
if frosting.use_background_gaussians:
    print(f"Number of Gaussians in the background: {len(frosting._bg_points)}")

# Save image

In [2]:
def save_image(tensor, path):
    if tensor.shape[0] == 3:
        tensor = tensor.transpose(1, 2, 0)
    ndarr = (tensor.cpu().numpy() * 255).clip(0, 255).astype(np.uint8)
    im = Image.fromarray(ndarr)
    im.save(path)

# Experiment: Shelly 2M

In [3]:
OUTPUT_ROOT = './saved_images/'
os.makedirs(OUTPUT_ROOT, exist_ok=True)

In [4]:
SHELLY_SCENES = {"kitten", "khady", "horse", "fernvase", "pug", "woolly"}
SHELLY_CAMERAS = {
    "kitten": [9, 15, 21, 26],
    "khady": [9, 16, 25, 29],
    "horse": [7, 8, 22, 29],
    "fernvase": [4, 11, 24, 26],
    "pug": [7, 16, 21, 24],
    "woolly": [0, 5, 7, 24]
}

In [5]:
_frostingfine = "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss2000000_frostlevel001_proposal30"

EXPERIMENTS_SHELLY_2M = {
    "gs_output_dir": "./output/vanilla_gs/{scene}/",
    "frosting_base": "./output/refined_frosting_std/{scene}/{_frostingfine}/15000.pt",
    "frosting_adap": "./output/refined_frosting/{scene}/{_frostingfine}/15000.pt"
}

In [10]:
eval_split = False
load_gt_images = True
white_background = True
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]
use_occlusion_culling = False

for scene in SHELLY_SCENES:
    source_path= f'./data/{scene}/'

    gs_output_dir = EXPERIMENTS_SHELLY_2M["gs_output_dir"].format(scene=scene)
    frosting_base_dir = EXPERIMENTS_SHELLY_2M["frosting_base"].format(scene=scene, _frostingfine=_frostingfine)
    frosting_adap_dir = EXPERIMENTS_SHELLY_2M["frosting_adap"].format(scene=scene, _frostingfine=_frostingfine)

    gs_model = GaussianSplattingWrapper(
        source_path=source_path,
        output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
        iteration_to_load=7000,
        load_gt_images=load_gt_images,
        eval_split=eval_split,
        eval_split_interval=8,
        background=bg_color,
        white_background=white_background,
        remove_camera_indices=[],
    )

    gt_imgs = []
    gs_cam_infos = []
    gs_imgs = []
    fr_base_cam_infos = []
    fr_base_imgs = []
    fr_adap_cam_infos = []
    fr_adap_imgs = []

    for cam_idx in SHELLY_CAMERAS[scene]:
        gs_cam_infos.append(gs_model.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render image with the Gaussian Splatting model
            rgb_img = gs_model.render_image(camera_indices=cam_idx).clamp(0,1)
            
            # Get GT image if available
            if load_gt_images:
                gt_img = gs_model.get_gt_image(cam_idx)
            else:
                gt_img = torch.zeros_like(rgb_img)
        gt_imgs.append(gt_img)
        gs_imgs.append(rgb_img)

    # FROSTING BASE
    frosting_base = load_frosting_model(frosting_base_dir, nerfmodel=gs_model)
    for cam_idx in SHELLY_CAMERAS[scene]:
        fr_base_cam_infos.append(frosting_base.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_base.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_base_imgs.append(rgb_img)

    del frosting_base

    # FROSTING ADAP
    frosting_adap = load_frosting_model(frosting_adap_dir, nerfmodel=gs_model)
    for cam_idx in SHELLY_CAMERAS[scene]:
        fr_adap_cam_infos.append(frosting_adap.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_adap.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_adap_imgs.append(rgb_img)

    del frosting_adap
    del gs_model

    torch.cuda.empty_cache()

    # Ensure that the camera infos match
    assert gs_cam_infos == fr_base_cam_infos == fr_adap_cam_infos, "Camera infos do not match!"
    cam_infos = gs_cam_infos
    # Save images
    scene_output_dir = os.path.join(OUTPUT_ROOT, scene, 'vs')
    os.makedirs(scene_output_dir, exist_ok=True)
    for i, cam_info in enumerate(cam_infos):
        save_image(gt_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_gt.png"))
        save_image(gs_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_gs.png"))
        save_image(fr_base_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_base.png"))
        save_image(fr_adap_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_adap.png"))

Found image extension .png
Creating Frosting model from checkpoint: ./output/refined_frosting_std/woolly/frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss2000000_frostlevel001_proposal30/15000.pt
[WARNING] No background Gaussians found.
No coarse sugar model provided. The frosting will not be initialized.

 [GDA ADAPTIVE ANALYSIS - ROBUST]
 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
 > Scene Scale (Diag):    5.9345
 > Relative Mean:         0.001950
 > Relative Std:          0.000000
 > Fuzziness Score:       0.001950 (Mean + Std)

GDA PROJECT - Initialization Strategy:
   > Force Exact Budget: False
   > Target Budget: 2000000
   > Ratio Uniform: 0.5
   > Total cell count: 583160
   > Mode: LEGACY (Guarantees coverage, may increase budget)
   > Legacy n_min per cell: 2
   > Final Actual Gaussian Count: 2000000
Frosting loaded.
Creating Frosting model from checkpoint: ./output/refined_frosting/woolly/frostingfine_3Dgs7000_de

# Experiment: Extreme

In [6]:
FROSTINGFINES = {
    "kitten": "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss350000_frostlevel001_proposal30",
    "khady": "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss150000_frostlevel001_proposal30",
    "pug": "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss200000_frostlevel001_proposal30",
    "horse": "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss200000_frostlevel001_proposal30",
    "fernvase": "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss150000_frostlevel001_proposal30",
    "woolly": "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss800000_frostlevel001_proposal30",
}

EXPERIMENT_EXTREME = {
    "gs_output_dir": "./output/vanilla_gs/{scene}/",
    "frosting_base": "./output/refined_frosting_extreme_base/{scene}/{_frostingfine}/15000.pt",
    "frosting_adap": "./output/refined_frosting_extreme_adap/{scene}/{_frostingfine}/15000.pt"
}

In [7]:
eval_split = False
load_gt_images = True
white_background = True
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]
use_occlusion_culling = False

for scene in SHELLY_SCENES:
    source_path= f'./data/{scene}/'
    _frostingfine = FROSTINGFINES[scene]

    gs_output_dir = EXPERIMENT_EXTREME["gs_output_dir"].format(scene=scene)
    frosting_base_dir = EXPERIMENT_EXTREME["frosting_base"].format(scene=scene, _frostingfine=_frostingfine)
    frosting_adap_dir = EXPERIMENT_EXTREME["frosting_adap"].format(scene=scene, _frostingfine=_frostingfine)

    gs_model = GaussianSplattingWrapper(
        source_path=source_path,
        output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
        iteration_to_load=7000,
        load_gt_images=load_gt_images,
        eval_split=eval_split,
        eval_split_interval=8,
        background=bg_color,
        white_background=white_background,
        remove_camera_indices=[],
    )

    
    gs_cam_infos = []
    fr_base_cam_infos = []
    fr_base_imgs = []
    fr_adap_cam_infos = []
    fr_adap_imgs = []

    for cam_idx in SHELLY_CAMERAS[scene]:
        gs_cam_infos.append(gs_model.training_cameras.gs_cameras[cam_idx].image_name)

    # FROSTING BASE
    frosting_base = load_frosting_model(frosting_base_dir, nerfmodel=gs_model)
    for cam_idx in SHELLY_CAMERAS[scene]:
        fr_base_cam_infos.append(frosting_base.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_base.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_base_imgs.append(rgb_img)

    del frosting_base

    # FROSTING ADAP
    frosting_adap = load_frosting_model(frosting_adap_dir, nerfmodel=gs_model)
    for cam_idx in SHELLY_CAMERAS[scene]:
        fr_adap_cam_infos.append(frosting_adap.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_adap.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_adap_imgs.append(rgb_img)

    del frosting_adap
    del gs_model

    torch.cuda.empty_cache()

    # Ensure that the camera infos match
    assert gs_cam_infos == fr_base_cam_infos == fr_adap_cam_infos, "Camera infos do not match!"
    cam_infos = gs_cam_infos
    # Save images
    scene_output_dir = os.path.join(OUTPUT_ROOT, scene, 'extreme')
    os.makedirs(scene_output_dir, exist_ok=True)
    for i, cam_info in enumerate(cam_infos):
        save_image(fr_base_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_base.png"))
        save_image(fr_adap_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_adap.png"))

Found image extension .png
Creating Frosting model from checkpoint: ./output/refined_frosting_extreme_base/kitten/frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss350000_frostlevel001_proposal30/15000.pt
[WARNING] No background Gaussians found.
No coarse sugar model provided. The frosting will not be initialized.

 [GDA ADAPTIVE ANALYSIS - ROBUST]
 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
 > Scene Scale (Diag):    2.8597
 > Relative Mean:         0.002698
 > Relative Std:          0.000000
 > Fuzziness Score:       0.002698 (Mean + Std)

GDA PROJECT - Initialization Strategy:
   > Force Exact Budget: False
   > Target Budget: 350000
   > Ratio Uniform: 0.5
   > Total cell count: 244529
   > Mode: LEGACY (Guarantees coverage, may increase budget)
   > Legacy n_min per cell: 1
   > Final Actual Gaussian Count: 350000
Frosting loaded.
Creating Frosting model from checkpoint: ./output/refined_frosting_extreme_adap/kitten/frost

# Experiment: Short time

In [8]:
_frostingfine = "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss1000000_frostlevel001_proposal30"

EXPERIMENT_FAST = {
    "gs_output_dir": "./output/vanilla_gs/{scene}/",
    "frosting_base": "./output/refined_frosting_fast_base/{scene}/{_frostingfine}/2000.pt",
    "frosting_adap": "./output/refined_frosting_fast_adap/{scene}/{_frostingfine}/2000.pt"
}

In [10]:
eval_split = False
load_gt_images = True
white_background = True
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]
use_occlusion_culling = False

for scene in SHELLY_SCENES:
    source_path= f'./data/{scene}/'

    gs_output_dir = EXPERIMENT_FAST["gs_output_dir"].format(scene=scene)
    frosting_base_dir = EXPERIMENT_FAST["frosting_base"].format(scene=scene, _frostingfine=_frostingfine)
    frosting_adap_dir = EXPERIMENT_FAST["frosting_adap"].format(scene=scene, _frostingfine=_frostingfine)

    gs_model = GaussianSplattingWrapper(
        source_path=source_path,
        output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
        iteration_to_load=7000,
        load_gt_images=load_gt_images,
        eval_split=eval_split,
        eval_split_interval=8,
        background=bg_color,
        white_background=white_background,
        remove_camera_indices=[],
    )

    gs_cam_infos = []
    fr_base_cam_infos = []
    fr_base_imgs = []
    fr_adap_cam_infos = []
    fr_adap_imgs = []

    for cam_idx in SHELLY_CAMERAS[scene]:
        gs_cam_infos.append(gs_model.training_cameras.gs_cameras[cam_idx].image_name)

    # FROSTING BASE
    frosting_base = load_frosting_model(frosting_base_dir, nerfmodel=gs_model)
    for cam_idx in SHELLY_CAMERAS[scene]:
        fr_base_cam_infos.append(frosting_base.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_base.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_base_imgs.append(rgb_img)

    del frosting_base

    # FROSTING ADAP
    frosting_adap = load_frosting_model(frosting_adap_dir, nerfmodel=gs_model)
    for cam_idx in SHELLY_CAMERAS[scene]:
        fr_adap_cam_infos.append(frosting_adap.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_adap.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_adap_imgs.append(rgb_img)

    del frosting_adap
    del gs_model

    torch.cuda.empty_cache()

    # Ensure that the camera infos match
    assert gs_cam_infos == fr_base_cam_infos == fr_adap_cam_infos, "Camera infos do not match!"
    cam_infos = gs_cam_infos
    # Save images
    scene_output_dir = os.path.join(OUTPUT_ROOT, scene, 'short_time')
    os.makedirs(scene_output_dir, exist_ok=True)
    for i, cam_info in enumerate(cam_infos):
        save_image(fr_base_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_base.png"))
        save_image(fr_adap_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_adap.png"))

Found image extension .png
Creating Frosting model from checkpoint: ./output/refined_frosting_fast_base/kitten/frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss1000000_frostlevel001_proposal30/2000.pt
[WARNING] No background Gaussians found.
No coarse sugar model provided. The frosting will not be initialized.

 [GDA ADAPTIVE ANALYSIS - ROBUST]
 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
 > Scene Scale (Diag):    2.8597
 > Relative Mean:         0.002698
 > Relative Std:          0.000000
 > Fuzziness Score:       0.002698 (Mean + Std)

GDA PROJECT - Initialization Strategy:
   > Force Exact Budget: False
   > Target Budget: 1000000
   > Ratio Uniform: 0.5
   > Total cell count: 244529
   > Mode: LEGACY (Guarantees coverage, may increase budget)
   > Legacy n_min per cell: 3
   > Final Actual Gaussian Count: 1000000
Frosting loaded.
Creating Frosting model from checkpoint: ./output/refined_frosting_fast_adap/kitten/frostingf

# Stick 2M

In [22]:
_frostingfine = "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile01_gauss2000000_frostlevel001_proposal30"
_scene = "stick"
_stick_cameras = [6, 10, 12, 16, 21, 22]

EXPERIMENT_STICK_2M = {
    "gs_output_dir": "./output/vanilla_gs/{scene}/",
    "frosting_base": "./output/refined_frosting/{scene}_std/{_frostingfine}/15000.pt",
    "frosting_adap": "./output/refined_frosting/{scene}/{_frostingfine}/15000.pt"
}

In [23]:
eval_split = True
load_gt_images = True
white_background = False
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]

if True:

    source_path= f'./own_data/{_scene}/'
    gs_output_dir = EXPERIMENT_STICK_2M["gs_output_dir"].format(scene=_scene)
    frosting_base_dir = EXPERIMENT_STICK_2M["frosting_base"].format(scene=_scene, _frostingfine=_frostingfine)
    frosting_adap_dir = EXPERIMENT_STICK_2M["frosting_adap"].format(scene=_scene, _frostingfine=_frostingfine)

    gs_model = GaussianSplattingWrapper(
        source_path=source_path,
        output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
        iteration_to_load=7000,
        load_gt_images=load_gt_images,
        eval_split=eval_split,
        eval_split_interval=8,
        background=bg_color,
        white_background=white_background,
        remove_camera_indices=[],
    )

    gt_imgs = []
    gs_cam_infos = []
    gs_imgs = []
    fr_base_cam_infos = []
    fr_base_imgs = []
    fr_adap_cam_infos = []
    fr_adap_imgs = []

    for cam_idx in _stick_cameras:
        gs_cam_infos.append(gs_model.test_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render image with the Gaussian Splatting model
            rgb_img = gs_model.render_image(camera_indices=cam_idx).clamp(0,1)
            
            # Get GT image if available
            if load_gt_images:
                gt_img = gs_model.get_gt_image(cam_idx)
            else:
                gt_img = torch.zeros_like(rgb_img)
        gt_imgs.append(gt_img)
        gs_imgs.append(rgb_img)

    # FROSTING BASE
    frosting_base = load_frosting_model(frosting_base_dir, nerfmodel=gs_model)
    for cam_idx in _stick_cameras:
        fr_base_cam_infos.append(frosting_base.nerfmodel.test_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_base.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_base_imgs.append(rgb_img)

    del frosting_base

    # FROSTING ADAP
    frosting_adap = load_frosting_model(frosting_adap_dir, nerfmodel=gs_model)
    for cam_idx in _stick_cameras:
        fr_adap_cam_infos.append(frosting_adap.nerfmodel.test_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_adap.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_adap_imgs.append(rgb_img)

    del frosting_adap
    del gs_model

    torch.cuda.empty_cache()

    # Ensure that the camera infos match
    assert gs_cam_infos == fr_base_cam_infos == fr_adap_cam_infos, "Camera infos do not match!"
    cam_infos = gs_cam_infos
    # Save images
    scene_output_dir = os.path.join(OUTPUT_ROOT, _scene, '2M_gauss')
    os.makedirs(scene_output_dir, exist_ok=True)
    for i, cam_info in enumerate(cam_infos):
        save_image(gt_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_gt.png"))
        save_image(gs_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_gs.png"))
        save_image(fr_base_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_base.png"))
        save_image(fr_adap_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_adap.png"))

Found image extension .jpg
Creating Frosting model from checkpoint: ./output/refined_frosting/stick_std/frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile01_gauss2000000_frostlevel001_proposal30/15000.pt
No coarse sugar model provided. The frosting will not be initialized.

 [GDA ADAPTIVE ANALYSIS - ROBUST]
 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
 > Scene Scale (Diag):    17.7409
 > Relative Mean:         0.000351
 > Relative Std:          0.000000
 > Fuzziness Score:       0.000351 (Mean + Std)

GDA PROJECT - Initialization Strategy:
   > Force Exact Budget: False
   > Target Budget: 2000000
   > Ratio Uniform: 0.5
   > Total cell count: 863715
   > Mode: LEGACY (Guarantees coverage, may increase budget)
   > Legacy n_min per cell: 2
   > Final Actual Gaussian Count: 2000000
Frosting loaded.
Creating Frosting model from checkpoint: ./output/refined_frosting/stick/frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim100000

# Stick 5M

In [24]:
_frostingfine = "frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile01_gauss5000000_frostlevel001_proposal30"

_scene = "stick"
_stick_cameras = [6, 10, 12, 16, 21, 22]

EXPERIMENT_STICK_5M = {
    "gs_output_dir": "./output/vanilla_gs/{scene}/",
    "frosting_base": "./output/refined_frosting_stick_base/{scene}/{_frostingfine}/15000.pt",
    "frosting_adap": "./output/refined_frosting_stick_adap/{scene}/{_frostingfine}/15000.pt"
}

In [25]:
eval_split = True
load_gt_images = True
white_background = False
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]

if True:

    source_path= f'./own_data/{_scene}/'
    gs_output_dir = EXPERIMENT_STICK_5M["gs_output_dir"].format(scene=_scene)
    frosting_base_dir = EXPERIMENT_STICK_5M["frosting_base"].format(scene=_scene, _frostingfine=_frostingfine)
    frosting_adap_dir = EXPERIMENT_STICK_5M["frosting_adap"].format(scene=_scene, _frostingfine=_frostingfine)

    gs_model = GaussianSplattingWrapper(
        source_path=source_path,
        output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
        iteration_to_load=7000,
        load_gt_images=load_gt_images,
        eval_split=eval_split,
        eval_split_interval=8,
        background=bg_color,
        white_background=white_background,
        remove_camera_indices=[],
    )

    gs_cam_infos = []
    fr_base_cam_infos = []
    fr_base_imgs = []
    fr_adap_cam_infos = []
    fr_adap_imgs = []

    for cam_idx in _stick_cameras:
        gs_cam_infos.append(gs_model.test_cameras.gs_cameras[cam_idx].image_name)

    # FROSTING BASE
    frosting_base = load_frosting_model(frosting_base_dir, nerfmodel=gs_model)
    for cam_idx in _stick_cameras:
        fr_base_cam_infos.append(frosting_base.nerfmodel.test_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_base.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_base_imgs.append(rgb_img)

    del frosting_base

    # FROSTING ADAP
    frosting_adap = load_frosting_model(frosting_adap_dir, nerfmodel=gs_model)
    for cam_idx in _stick_cameras:
        fr_adap_cam_infos.append(frosting_adap.nerfmodel.test_cameras.gs_cameras[cam_idx].image_name)
        with torch.no_grad():
            # Render RGB image
            rgb_img = frosting_adap.render_image_gaussian_rasterizer(
                camera_indices=cam_idx,
                compute_color_in_rasterizer=True,
                use_occlusion_culling=use_occlusion_culling,
                bg_color=torch.tensor(bg_color, device=device),
            ).clamp(0, 1)
        fr_adap_imgs.append(rgb_img)

    del frosting_adap
    del gs_model

    torch.cuda.empty_cache()

    # Ensure that the camera infos match
    assert gs_cam_infos == fr_base_cam_infos == fr_adap_cam_infos, "Camera infos do not match!"
    cam_infos = gs_cam_infos
    # Save images
    scene_output_dir = os.path.join(OUTPUT_ROOT, _scene, '5M_gauss')
    os.makedirs(scene_output_dir, exist_ok=True)
    for i, cam_info in enumerate(cam_infos):
        save_image(gt_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_gt.png"))
        save_image(gs_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_gs.png"))
        save_image(fr_base_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_base.png"))
        save_image(fr_adap_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr_adap.png"))

Found image extension .jpg
Creating Frosting model from checkpoint: ./output/refined_frosting_stick_base/stick/frostingfine_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile01_gauss5000000_frostlevel001_proposal30/15000.pt
No coarse sugar model provided. The frosting will not be initialized.

 [GDA ADAPTIVE ANALYSIS - ROBUST]
 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
 > Scene Scale (Diag):    17.7409
 > Relative Mean:         0.000351
 > Relative Std:          0.000000
 > Fuzziness Score:       0.000351 (Mean + Std)

GDA PROJECT - Initialization Strategy:
   > Force Exact Budget: False
   > Target Budget: 5000000
   > Ratio Uniform: 0.5
   > Total cell count: 863715
   > Mode: LEGACY (Guarantees coverage, may increase budget)
   > Legacy n_min per cell: 3
   > Final Actual Gaussian Count: 5000000
Frosting loaded.
Creating Frosting model from checkpoint: ./output/refined_frosting_stick_adap/stick/frostingfine_3Dgs7000_densityestim02_sdfnorm02_l

# For differences (divergence...)

In [31]:
_scene = "khady"
source_path= f'./data/{_scene}/'
gs_output_dir = f"./output/vanilla_gs/{_scene}/"
RATIOS = [0.0, 0.25, 0.5, 0.75, 1.0]
MODELS_ROOT = f"./output/refined_frosting_std/{_scene}/"
N_GAUSSIANS = 10000
CHECKPOINT = "15000.pt"

In [33]:
eval_split = False
load_gt_images = True
white_background = True
bg_color = [1., 1., 1.] if white_background else [0., 0., 0.]
use_occlusion_culling = False

if True:
    gs_model = GaussianSplattingWrapper(
        source_path=source_path,
        output_path=gs_output_dir if gs_output_dir.endswith(os.sep) else gs_output_dir + os.sep,
        iteration_to_load=7000,
        load_gt_images=load_gt_images,
        eval_split=eval_split,
        eval_split_interval=8,
        background=bg_color,
        white_background=white_background,
        remove_camera_indices=[],
    )
    
    gs_cam_infos = []
    fr_base_cam_infos = []
    fr_base_imgs = []
    fr_adap_cam_infos = []
    fr_adap_imgs = []

    for cam_idx in SHELLY_CAMERAS[_scene]:
        gs_cam_infos.append(gs_model.training_cameras.gs_cameras[cam_idx].image_name)

    for ratio in RATIOS:
        print(f"> Processing Ratio {ratio}...")
        fr_cam_infos = []
        fr_imgs = []
        
        # Pattern matching
        ratio_str = str(ratio).replace('.', '')
        if len(ratio_str) == 1: ratio_str += '0'
        
        search_pattern = os.path.join(MODELS_ROOT, f"*unifSampRatio{ratio_str}_*_gauss{N_GAUSSIANS}_*")
        candidates = glob.glob(search_pattern)
        
        if not candidates:
            print(f"   [Warning] Model for ratio {ratio} not found. Skipping.")
            continue
            
        model_path = os.path.join(candidates[0], CHECKPOINT)
        print(f"   Found model at: {model_path}")

        try:
            # Load Model
            frosting_ratio = load_frosting_model(
                model_path, 
                nerfmodel=gs_model, 
                device=device, 
                uniform_sampling_ratio=ratio,
                force_exact_gaussians_number=True
            )
            frosting_ratio.eval()
        except Exception as e:
            print(f"   [Error] Failed ratio {ratio}: {e}")

        for cam_idx in SHELLY_CAMERAS[_scene]:
            fr_cam_infos.append(frosting_ratio.nerfmodel.training_cameras.gs_cameras[cam_idx].image_name)

            with torch.no_grad():
                render_pkg = frosting_ratio.render_image_gaussian_rasterizer(
                    camera_indices=cam_idx,
                    bg_color=torch.tensor(bg_color, device=device),
                    compute_color_in_rasterizer=True,
                    use_occlusion_culling=use_occlusion_culling
                ).clamp(0, 1)
            fr_imgs.append(render_pkg)

        # Ensure that the camera infos match
        assert gs_cam_infos == fr_cam_infos, "Camera infos do not match!"
        cam_infos = gs_cam_infos
        # Save images
        scene_output_dir = os.path.join(OUTPUT_ROOT, _scene, f'ratio_{ratio_str}')
        os.makedirs(scene_output_dir, exist_ok=True)
        for i, cam_info in enumerate(cam_infos):
            save_image(fr_imgs[i], os.path.join(scene_output_dir, f"cam_{cam_info}_fr.png"))
        
        del frosting_ratio

del gs_model
torch.cuda.empty_cache()

Found image extension .png
> Processing Ratio 0.0...
   Found model at: ./output/refined_frosting_std/khady/frostingfine_unifSampRatio00_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss10000_frostlevel001_proposal30/15000.pt
Creating Frosting model from checkpoint: ./output/refined_frosting_std/khady/frostingfine_unifSampRatio00_3Dgs7000_densityestim02_sdfnorm02_level03_decim1000000_depthauto_quantile00_gauss10000_frostlevel001_proposal30/15000.pt
No coarse sugar model provided. The frosting will not be initialized.

 [GDA ADAPTIVE ANALYSIS - ROBUST]
 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
 > Scene Scale (Diag):    2.5066
 > Relative Mean:         0.002181
 > Relative Std:          0.000000
 > Fuzziness Score:       0.002181 (Mean + Std)

GDA PROJECT - Initialization Strategy:
   > Force Exact Budget: True
   > Target Budget: 10000
   > Ratio Uniform: 0.0
   > Total cell count: 96917
   > Mode: STRICT (Allowing empty cells to respec